In [1]:
from tensorflow.python.lib.io.file_io import FileIO as open_file
import pandas as pd
import json
import math

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DATASET_CSV="gs://ml-research-injenia/estimators/datasets/kickstarter-set/ks-projects-201612.csv"
OUT_DIR="gs://ml-research-injenia/estimators/datasets/kickstarter-set"
with open_file(DATASET_CSV, "r") as f:
    original_df=pd.read_csv(f)
original_df

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN
5,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 18:30:44,1000,2014-12-01 18:30:44,1205,successful,16,US,1205,NaN,NaN,NaN,NaN
6,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17 19:05:12,25000,2016-02-01 20:05:12,453,failed,40,US,453,NaN,NaN,NaN,NaN
7,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29 18:14:43,125000,2014-04-24 18:14:43,8233,canceled,58,US,8233,NaN,NaN,NaN,NaN
8,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10 21:55:48,65000,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,NaN,NaN,NaN,NaN
9,100004721,Of Jesus and Madmen,Nonfiction,Publishing,CAD,2013-10-09 18:19:37,2500,2013-09-09 18:19:37,0,failed,0,CA,0,NaN,NaN,NaN,NaN


In [4]:
df=original_df.copy()
df=df.rename(columns={x:x.strip() for x in df.columns})

In [6]:
df[df["state"].isin(["failed","successful"])]["state"].value_counts()

failed        168221
successful    113081
Name: state, dtype: int64

In [8]:
from __future__ import division
168221/(168221+113081),113081/(168221+113081)

(0.5980085459754997, 0.40199145402450037)

In [140]:
SELECTED_COLUMNS=["ID","category","main_category","currency","deadline","launched","goal","state","country"]
df=df[SELECTED_COLUMNS]

In [141]:
df["timespan"]=pd.to_datetime(df["deadline"],errors="coerce")-pd.to_datetime(df["launched"],errors="coerce")
df

,ID,category,main_category,currency,deadline,launched,goal,state,country,timespan
0,1000002330,Poetry,Publishing,GBP,2015-10-09 11:36:00,2015-08-11 12:12:28,1000,failed,GB,58 days 23:23:32
1,1000004038,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,2013-01-12 00:20:50,45000,failed,US,45 days 00:00:00
2,1000007540,Music,Music,USD,2012-04-16 04:24:11,2012-03-17 03:24:11,5000,failed,US,30 days 01:00:00
3,1000011046,Film & Video,Film & Video,USD,2015-08-29 01:00:00,2015-07-04 08:35:03,19500,canceled,US,55 days 16:24:57
4,1000014025,Restaurants,Food,USD,2016-04-01 13:38:27,2016-02-26 13:38:27,50000,successful,US,35 days 00:00:00
5,1000023410,Food,Food,USD,2014-12-21 18:30:44,2014-12-01 18:30:44,1000,successful,US,20 days 00:00:00
6,1000030581,Drinks,Food,USD,2016-03-17 19:05:12,2016-02-01 20:05:12,25000,failed,US,44 days 23:00:00
7,1000034518,Product Design,Design,USD,2014-05-29 18:14:43,2014-04-24 18:14:43,125000,canceled,US,35 days 00:00:00
8,100004195,Documentary,Film & Video,USD,2014-08-10 21:55:48,2014-07-11 21:55:48,65000,canceled,US,30 days 00:00:00
9,100004721,Nonfiction,Publishing,CAD,2013-10-09 18:19:37,2013-09-09 18:19:37,2500,failed,CA,30 days 00:00:00


In [142]:
df["timespan_days"]=pd.to_timedelta(df["timespan"]).astype('timedelta64[D]')
df=df[~df["timespan_days"].isna()]

In [143]:
df["goal"]=pd.to_numeric(df["goal"], errors='coerce')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [144]:
df.isna().any()

ID               False
category         False
main_category    False
currency         False
deadline         False
launched         False
goal             False
state            False
country          False
timespan         False
timespan_days    False
dtype: bool

In [145]:
df=df[df["state"].isin(["failed","successful"])]

In [146]:
df["currency"].value_counts()

USD    229517
GBP     23842
CAD      9904
EUR      9177
AUD      5188
SEK      1066
NZD       992
DKK       675
NOK       432
CHF       389
SGD        69
HKD        47
MXN         4
Name: currency, dtype: int64

In [147]:
from currency_converter import CurrencyConverter
c = CurrencyConverter()

In [148]:
converted=[c.convert(x[0], x[1], 'USD') for x in df[["goal","currency"]].values]
converted[:5]

[1403.5561861648819, 45000.0, 5000.0, 50000.0, 1000.0]

In [149]:
df["goal_USD"]=converted

In [161]:
df["goal_USD_log"]=df["goal_USD"].apply(math.log)
df

,ID,category,main_category,currency,deadline,launched,goal,state,country,timespan,timespan_days,goal_USD,goal_USD_log
0,1000002330,Poetry,Publishing,GBP,2015-10-09 11:36:00,2015-08-11 12:12:28,1000.0,failed,GB,58 days 23:23:32,58.0,1403.556186,7.246764
1,1000004038,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,2013-01-12 00:20:50,45000.0,failed,US,45 days 00:00:00,45.0,45000.000000,10.714418
2,1000007540,Music,Music,USD,2012-04-16 04:24:11,2012-03-17 03:24:11,5000.0,failed,US,30 days 01:00:00,30.0,5000.000000,8.517193
4,1000014025,Restaurants,Food,USD,2016-04-01 13:38:27,2016-02-26 13:38:27,50000.0,successful,US,35 days 00:00:00,35.0,50000.000000,10.819778
5,1000023410,Food,Food,USD,2014-12-21 18:30:44,2014-12-01 18:30:44,1000.0,successful,US,20 days 00:00:00,20.0,1000.000000,6.907755
6,1000030581,Drinks,Food,USD,2016-03-17 19:05:12,2016-02-01 20:05:12,25000.0,failed,US,44 days 23:00:00,44.0,25000.000000,10.126631
9,100004721,Nonfiction,Publishing,CAD,2013-10-09 18:19:37,2013-09-09 18:19:37,2500.0,failed,CA,30 days 00:00:00,30.0,1999.101816,7.600453
10,100005484,Indie Rock,Music,USD,2013-04-08 06:42:58,2013-03-09 06:42:58,12500.0,successful,US,30 days 00:00:00,30.0,12500.000000,9.433484
11,1000055792,Crafts,Crafts,USD,2014-10-02 17:11:50,2014-09-02 17:11:50,5000.0,failed,US,30 days 00:00:00,30.0,5000.000000,8.517193
12,1000056157,Games,Games,USD,2016-03-25 22:01:12,2016-02-09 23:01:12,200000.0,failed,US,44 days 23:00:00,44.0,200000.000000,12.206073


In [162]:
NUMERICAL_COLUMNS=["timespan_days","goal_USD","goal_USD_log"]

In [163]:
df[NUMERICAL_COLUMNS].values

array([[5.80000000e+01, 1.40355619e+03, 7.24676443e+00],
       [4.50000000e+01, 4.50000000e+04, 1.07144178e+01],
       [3.00000000e+01, 5.00000000e+03, 8.51719319e+00],
       ...,
       [4.50000000e+01, 1.50000000e+04, 9.61580548e+00],
       [3.00000000e+01, 1.50000000e+04, 9.61580548e+00],
       [2.80000000e+01, 2.00000000e+03, 7.60090246e+00]])

In [164]:
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

In [171]:
scaler = StandardScaler()
scaler.fit(df[NUMERICAL_COLUMNS].values)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [172]:
scaler_params={
    "columns":NUMERICAL_COLUMNS,
    "mean":list(scaler.mean_),
    "var":list(scaler.var_),
    "scale":list(scaler.scale_)}
scaler_params

{'columns': ['timespan_days', 'goal_USD', 'goal_USD_log'],
 'mean': [33.972509971489714, 41056.15247426807, 8.581105517205506],
 'scale': [12.891265854695337, 1069994.523166219, 1.667602893135461],
 'var': [166.1847353364339, 1144888279605.7043, 2.7808994091937596]}

In [173]:
with open_file(OUT_DIR+"/2016_scaler.pkl", "w") as f:
    joblib.dump(scaler , f)
with open_file(OUT_DIR+"/2016_scaler.json", "w") as f:
    json.dump(scaler_params , f)

In [174]:
scaled=scaler.transform(df[NUMERICAL_COLUMNS].values)
scaled[:5]

array([[ 1.86385808, -0.03705869, -0.80015518],
       [ 0.85542337,  0.00368586,  1.27926874],
       [-0.30815515, -0.03369751, -0.03832587],
       [ 0.07970435,  0.00835878,  1.34244956],
       [-1.08387416, -0.03743585, -1.00344647]])

In [175]:
for i,c in enumerate(NUMERICAL_COLUMNS):
    df[c+"_scaled"]=[x[i] for x in scaled]

In [176]:
df

,ID,category,main_category,currency,deadline,launched,goal,state,country,timespan,timespan_days,goal_USD,goal_USD_log,timespan_days_scaled,goal_USD_scaled,goal_USD_log_scaled
0,1000002330,Poetry,Publishing,GBP,2015-10-09 11:36:00,2015-08-11 12:12:28,1000.0,failed,GB,58 days 23:23:32,58.0,1403.556186,7.246764,1.863858,-0.037059,-0.800155
1,1000004038,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,2013-01-12 00:20:50,45000.0,failed,US,45 days 00:00:00,45.0,45000.000000,10.714418,0.855423,0.003686,1.279269
2,1000007540,Music,Music,USD,2012-04-16 04:24:11,2012-03-17 03:24:11,5000.0,failed,US,30 days 01:00:00,30.0,5000.000000,8.517193,-0.308155,-0.033698,-0.038326
4,1000014025,Restaurants,Food,USD,2016-04-01 13:38:27,2016-02-26 13:38:27,50000.0,successful,US,35 days 00:00:00,35.0,50000.000000,10.819778,0.079704,0.008359,1.342450
5,1000023410,Food,Food,USD,2014-12-21 18:30:44,2014-12-01 18:30:44,1000.0,successful,US,20 days 00:00:00,20.0,1000.000000,6.907755,-1.083874,-0.037436,-1.003446
6,1000030581,Drinks,Food,USD,2016-03-17 19:05:12,2016-02-01 20:05:12,25000.0,failed,US,44 days 23:00:00,44.0,25000.000000,10.126631,0.777851,-0.015006,0.926795
9,100004721,Nonfiction,Publishing,CAD,2013-10-09 18:19:37,2013-09-09 18:19:37,2500.0,failed,CA,30 days 00:00:00,30.0,1999.101816,7.600453,-0.308155,-0.036502,-0.588061
10,100005484,Indie Rock,Music,USD,2013-04-08 06:42:58,2013-03-09 06:42:58,12500.0,successful,US,30 days 00:00:00,30.0,12500.000000,9.433484,-0.308155,-0.026688,0.511140
11,1000055792,Crafts,Crafts,USD,2014-10-02 17:11:50,2014-09-02 17:11:50,5000.0,failed,US,30 days 00:00:00,30.0,5000.000000,8.517193,-0.308155,-0.033698,-0.038326
12,1000056157,Games,Games,USD,2016-03-25 22:01:12,2016-02-09 23:01:12,200000.0,failed,US,44 days 23:00:00,44.0,200000.000000,12.206073,0.777851,0.148546,2.173759


In [178]:
scaler2 = StandardScaler()
scaler2.fit(df[[c+"_scaled" for c in NUMERICAL_COLUMNS]].values)
{
    "mean":scaler2.mean_,
    "var":scaler2.var_,
    "scale":scaler2.scale_}

{'mean': array([1.09977998e-16, 6.01165904e-18, 3.00077771e-16]),
 'scale': array([1., 1., 1.]),
 'var': array([1., 1., 1.])}

In [181]:
dataset_df=df[[
    "ID",
    "category",
    "main_category",
    "currency",
    "state",
    "country",
    "timespan_days",
    "goal_USD",
    "goal_USD_log",
    "timespan_days_scaled",
    "goal_USD_scaled",
    "goal_USD_log_scaled"
]]

In [182]:
for c in dataset_df.columns:
    print(dataset_df[c].value_counts())

1294469119    1
1177862349    1
839094673     1
457474245     1
1679071430    1
674009287     1
1049422024    1
1574236363    1
560779471     1
1613194665    1
192690386     1
1199311060    1
536621219     1
559184090     1
1296324830    1
924168419     1
645177537     1
1812768960    1
1387221183    1
1226784957    1
1907837116    1
1914663099    1
675803556     1
539977912     1
71879904      1
468670148     1
904345778     1
89598128      1
1245696175    1
1529338030    1
             ..
2103139009    1
882070208     1
1918145406    1
1457029818    1
248727622     1
2018044599    1
253820204     1
194038451     1
1955496804    1
353013964     1
1074801327    1
1854717019    1
1281374893    1
986192553     1
1657914002    1
1730155176    1
1622172326    1
1007176355    1
1595517599    1
1192338078    1
754561692     1
215137044     1
832150169     1
1955931023    1
144304791     1
449438358     1
1276246677    1
181531284     1
588565706     1
959686544     1
Name: ID, Length: 281302

In [184]:
with open_file(OUT_DIR+"/2016_dataset.csv","w") as f:
    dataset_df.to_csv(f,index=False)

### train and eval

In [202]:
LABEL_COLUMN="state"
eval_percent=0.1

In [186]:
labels=dataset_df[LABEL_COLUMN].unique()
labels

array(['failed', 'successful'], dtype=object)

In [198]:
individuals_df={l:dataset_df[dataset_df[LABEL_COLUMN]==l].sample(frac=1) for l in labels}

In [203]:
splitted_dfs={}
for l in labels:
    splitted_dfs[l]={
        "eval":individuals_df[l][:int(float(individuals_df[l].shape[0])*eval_percent)],
        "train":individuals_df[l][int(float(individuals_df[l].shape[0])*eval_percent):]
    }

In [204]:
train_set=pd.concat([splitted_dfs[l]["train"] for l in splitted_dfs]).sample(frac=1)
eval_set=pd.concat([splitted_dfs[l]["eval"] for l in splitted_dfs]).sample(frac=1)

In [205]:
train_set

,ID,category,main_category,currency,state,country,timespan_days,goal_USD,goal_USD_log,timespan_days_scaled,goal_USD_scaled,goal_USD_log_scaled
150183,1892252564,Comic Books,Comics,USD,successful,US,30.0,2500.000000,7.824046,-0.308155,-0.036034,-0.453981
95865,1569410360,Documentary,Film & Video,USD,failed,US,60.0,2000.000000,7.600902,2.019002,-0.036501,-0.587792
274495,703720557,Illustration,Art,USD,failed,US,30.0,2700.000000,7.901007,-0.308155,-0.035847,-0.407830
206435,296713470,Photography,Photography,USD,successful,US,30.0,5000.000000,8.517193,-0.308155,-0.033698,-0.038326
95002,156448762,Product Design,Design,USD,failed,US,30.0,15000.000000,9.615805,-0.308155,-0.024352,0.620471
43008,1255814383,Music,Music,USD,successful,US,30.0,100000.000000,11.512925,-0.308155,0.055088,1.758104
273463,697900694,Video Games,Games,CAD,failed,CA,30.0,1199.461089,7.089628,-0.308155,-0.037249,-0.894384
4527,1026864342,Civic Design,Design,EUR,failed,IT,60.0,24928.000000,10.123747,2.019002,-0.015073,0.925065
155283,1923292016,Farmer's Markets,Food,GBP,failed,GB,25.0,140.355619,4.944179,-0.696015,-0.038239,-2.180931
209980,317811128,Publishing,Publishing,USD,failed,US,29.0,15000.000000,9.615805,-0.385727,-0.024352,0.620471


In [206]:
eval_set

,ID,category,main_category,currency,state,country,timespan_days,goal_USD,goal_USD_log,timespan_days_scaled,goal_USD_scaled,goal_USD_log_scaled
89885,153415837,Flight,Technology,USD,failed,US,30.0,1.500000e+04,9.615805,-0.308155,-0.024352,0.620471
139352,1828026144,Fine Art,Photography,USD,failed,US,31.0,6.500001e+06,15.687313,-0.230583,6.036428,4.261331
310349,919600393,Children's Books,Publishing,USD,failed,US,32.0,2.000000e+03,7.600902,-0.153011,-0.036501,-0.587792
104591,1621230227,Comic Books,Comics,USD,failed,US,30.0,4.000000e+03,8.294050,-0.308155,-0.034632,-0.172137
316319,954788005,Theater,Theater,USD,successful,US,14.0,3.500000e+03,8.160518,-1.549306,-0.035099,-0.252211
16016,1094453155,Flight,Technology,USD,failed,US,53.0,1.000000e+04,9.210340,1.475999,-0.029025,0.377329
246620,536236122,Experimental,Film & Video,USD,successful,US,17.0,2.500000e+01,3.218876,-1.316590,-0.038347,-3.215532
180740,2076494684,Quilts,Crafts,USD,failed,US,28.0,1.800000e+03,7.495542,-0.463299,-0.036688,-0.650972
274870,70609801,Young Adult,Publishing,EUR,failed,DE,60.0,1.246400e+04,9.430600,2.019002,-0.026722,0.509410
45342,1269516548,Product Design,Design,USD,successful,US,30.0,7.500000e+03,8.922658,-0.308155,-0.031361,0.204817


In [207]:
with open_file(OUT_DIR+"/2016_trainset.csv","w") as f:
    train_set.to_csv(f,index=False)
with open_file(OUT_DIR+"/2016_evalset.csv","w") as f:
    eval_set.to_csv(f,index=False)